In [ ]:
import sys, subprocess, os
from pathlib import Path

# Colab Setup
if "google.colab" in sys.modules:
    print("Running in Google Colab. Installing dependencies...")
    subprocess.run(["pip", "install", "-q", "pandas", "numpy", "scikit-learn", "requests", "pydantic", "jsonschema", "plotly", "tqdm"])
    
    # Check for data
    if not (Path.cwd() / "data").exists():
        print("Data directory not found. Cloning repository...")
        subprocess.run(["git", "clone", "https://github.com/aire-program/aire-researcher-sandbox.git", "_repo"])
        
        # Move data and scripts to current directory
        if (Path("_repo/data").exists()):
            print("Moving data and scripts...")
            subprocess.run(["mv", "_repo/data", "."])
            subprocess.run(["mv", "_repo/scripts", "."])
            subprocess.run(["rm", "-rf", "_repo"])
        else:
            print("Warning: Data not found in cloned repo.")
    else:
        print("Data directory found.")


# REST API Client Example

**What**: Interact with a synthetic REST API to manage research data.

**Why**: Modern research often involves interacting with remote data services. Understanding REST patterns is a key skill.

**How**:
1. **Initialize the client**.
2. **Make GET requests** to retrieve data.
3. **Make POST requests** to submit data.

**Key Concept**: **REST** (Representational State Transfer) is a standard architectural style for creating web services.

By the end of this notebook, you will have completed the listed steps and produced the outputs described in the success criteria.

### Success criteria
- You called the synthetic client’s health, GET, and POST helpers.
- You saw JSON responses.
- You traced request parameters for future adaptation.

In [ ]:
import sys
from pathlib import Path

repo_root = Path.cwd()
for candidate in [repo_root, repo_root.parent, repo_root.parent.parent]:
    if (candidate / "api" / "python" / "client_rest_api.py").exists():
        sys.path.append(str(candidate))
        break

from api.python.client_rest_api import ResearchAPIClient


class DummyResponse:
    def __init__(self, payload):
        self._payload = payload

    def raise_for_status(self):
        return None

    def json(self):
        return self._payload


class DummySession:
    def __init__(self):
        self.last_request = None

    def request(self, method: str, url: str, timeout: int = 5, **kwargs):
        self.last_request = {"method": method, "url": url, "kwargs": kwargs, "timeout": timeout}
        if url.endswith("/projects"):
            payload = {"projects": [
                {"project_id": "PRJ-001", "title": "Synthetic Study A"},
                {"project_id": "PRJ-002", "title": "Synthetic Study B"},
            ]}
        elif url.endswith("/datasets"):
            dataset_id = (kwargs.get("json") or {}).get("dataset_id", "UNKNOWN")
            payload = {"dataset_id": dataset_id, "schema_version": "v1", "status": "described"}
        else:
            payload = {"status": "ok", "method": method, "url": url}
        return DummyResponse(payload)


client = ResearchAPIClient(base_url="https://example.test", session=DummySession())
health = client.health()
health


## Retrieve synthetic projects

Projects are returned from the dummy session without real network calls.

In [ ]:
projects = client.get_json("/projects", params={"limit": 3})
projects


## Fetch dataset metadata

POST helpers echo synthetic dataset metadata using the dummy session.

In [ ]:
dataset = client.post_json("/datasets", payload={"dataset_id": "DATASET-001", "action": "describe"})
dataset


### If you get stuck / What to try next

If you get stuck: rerun dependency install and ensure repo root is on sys.path (handled in the notebook). What to try next: adapt payloads to your own endpoints or explore embeddings in api/notebooks/embeddings_api_example.ipynb.